# HW3_programQuestion

**Due to 11:59 pm, 28th, October 2020**

**This is an individual assignment.**

In [6]:
"""
Import libraries that you might require.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import LogisticRegression

## Logistic Regression and Gradient Descent

In this question, we will try to use logistic regression to solve a binary classification problem. Given some information of the house, such as area and the number of living rooms, would it be expensive? We would like to predict 1 if it is expensive, and 0 otherwise. 

We will first implement it with a python package, and then try to implement it by updating weights with gradient descent.Batch gradient descent (since we are using all samples at each iteration) and AdaGrad will be implemented. We will also derive the gradient formula.

### a) Implement logistic regression with Scikit learn package. 

First load data and observe data.

In [7]:
"""
Reads the data.
"""

X_train = pd.read_csv('hw3_house_sales/X_train.csv')
X_test = pd.read_csv('hw3_house_sales/X_test.csv')
y_train = pd.read_csv('hw3_house_sales/y_train.csv')
y_test = pd.read_csv('hw3_house_sales/y_test.csv')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(X_test.head(5))
print(y_test.head(5))

(800, 11)
(400, 11)
(800, 1)
(400, 1)
   LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  GrLivArea  \
0    11851            7            5       1990          1990       1442   
1     9920            7            5       1996          1997       2013   
2    11216            8            5       2006          2006       1489   
3     8244            7            5       2004          2004       1720   
4    11787            7            5       1996          1997       2398   

   FullBath  BedroomAbvGr  KitchenAbvGr  YrSold  const  
0         2             3             1    2009      1  
1         2             3             1    2007      1  
2         2             3             1    2006      1  
3         2             3             1    2007      1  
4         2             3             1    2007      1  
   label
0      1
1      1
2      1
3      1
4      1


Fill in the logisticRegressionScikit() function. Report the weights, training accuracy, and the test accuracy.

In [8]:
def LogisticRegressionScikit(X_train, y_train, X_test, y_test):
    """
    Computes logistic regression with scikit-learn.
    
    Args:
        X_train: feature matrix of training set
        y_train: truth value of training set
        X_test: feature matrix of test set
        y_test: truth value of test set

    Returns:  
        w: numpy array of learned coefficients
        y_pred: numpy array of predicted labels for the test data
        score: accuracy of test data
    """
    clf = LogisticRegression(random_state=0).fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    score = clf.score(X_test,y_test)
    coef = clf.coef_
    return coef, y_pred, score


In [9]:

coef_scikit, y_pred_scikit, acc_scikit = LogisticRegressionScikit(X_train, y_train, X_test, y_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(acc_scikit))
print('logistic regression coefficient:', coef_scikit)

Accuracy of logistic regression classifier on test set: 0.95
logistic regression coefficient: [[ 3.30147860e-04  1.91758109e+00  1.01659015e+00  7.61657425e-02
   1.36741708e-03  7.05591471e-03  2.48682303e-01 -9.27372661e-01
  -4.52099059e-01 -9.03354524e-02 -2.16364255e-04]]


### b) Gradient derivation
Calculate the maximum likelihood estimation $L(w)=P(Y|X;w)$, then formulate the stochastic gradient ascent rule.

$h(X;w) = \frac{1}{1+\exp^{-w^TX}}$

$P(Y|X;w)=h(x;w)^Y(1-h(x;w))^{(1-y)}$

$L(w) = \prod \limits_{i=1}^n p(Y^i|X^i;w)^{Y^i}(1-h(X^i;w))^{1-Y^i}$

$l(w) = \ln L(w) = \sum \limits_{i= 1}^{n}Y^i \ln (h(X^i;w)) + (1-Y^i)\ln(1-h(X^i;w))$

$w_j := w_j + \Delta w_j = w_j - \eta \frac{\partial}{\partial w_j} l(w)$

$\frac{\partial}{\partial w_j}l(w) = \sum \limits_{i=1}^n(Y^i - h(X;w))x_j$

$w_{j+1} = w_j - \eta \sum \limits_{i = 1}^n(Y^i-h(X;w))x_j$

### c) Logistic regression with simple gradient descent
Fill in the LogisticRegressionSGD() function. To do that, two helper functions, sigmoid_activation() (to calculate the sigmoid function result), and model_optimize() (to calculate the gradient of w), will be needed. Both helper functions can be used in the following AdaGrad optimization function. Use a learning rate of $10^{−4}$, run with 2000 iterations. Report the weights and accuracy. Keep track of the accuracy every 100 iterations in the training set. It will be used later.

In [10]:
def sigmoid_activation(x):
    """
    Calculates the sigmoid function.
    
    Args:
        x: numpy array of input
        
    Returns:
        final_result: numpy array of sigmoid result
    """
    final_result = 1.0/(1+np.exp(-x))
    return final_result

**Remember to derive the gradient (Question 4.2), write down the weight update formula, and hand it in with your latex submission!**

In [11]:
def model_optimize(w, X, Y):
    """
    Calculates gradient of the weights.
    
    Args:
        X: numpy array of training samples
        Y: numpy array of training labels
        w: numpy array of weights
    Returns:
        dw: the gradient of the weights

    """
    h = sigmoid_activation(X*w)
    error = h - Y
    dw = X.T * error
    return dw

In [90]:
def LogisticRegressionSGD(w, X, Y, learning_rate, num_iterations):
    """
    Uses SGD to update weights for logistic regression.
    
    Args:       
        w: numpy array of initial weights
        X: numpy array of training samples
        Y: numpy array of training labels
        learning_rate: float learning rate to update w
        num_iterations: int number of iterations to update w
    
    Returns:  
        coeff: numpy array of weights after optimization
        accuracies: a list of accuracy at each hundred's iteration. With 2000 iterations, 
                    accuracies should be a list of size 20 
    """
    m,n = X.shape
    accuracies = []
    for i in range(n):
        dw = model_optimize(w,X[i],Y[i])
        w = w - learning_rate * dw
        if i % 100 == 0:
            print(w)
            print(X)
            Y_test = np.dot(w,X.T)
            print(Y_test)
            Error = Y_test - Y
            for i in range(len(Error)):
                if Error[i] == Y_test[i]:
                    accur += 1
            accur = accur / len(Error)
            accuracies.append(accur)
    test_Y = w*X
    coeff = w
    return coeff, accuracies

### d) Logistic regression with AdaGrad
 Fill in the LogisticRegressionAda() function. Use a learning rate of $10^{−4}$, run with 2000 iterations. Report the weights and accuracy. Keep track of the accuracy every 100 iterations in the training set. It will be used later.

In [91]:
def LogisticRegressionAda(w, X, Y, learning_rate, num_iterations):
    """
    Use AdaGrad to update weights.
    
    Args:       
        w: numpy array of initial weights
        X: numpy array of training samples
        Y: numpy array of training labels
        learning_rate: float learning rate to update w
        num_iterations: int number of iterations to update w
    
    Returns:   
        coeff: numpy array of weights after optimization
        accuracies: a list of accuracy at each hundred's iteration
    """
    accuracies = []

    return coeff, accuracies

We add a predict() function here to threshold probability prediction into binary classification

In [92]:
def predict(final_pred, m):
    """
    Predict labels from probability to 0/1 label, threshold 0.5.
    
    Args:
        final_pred: m x 1 vector, probabilty of each sample belonging to class 1
        m: number of samples
        
    Returns:
        y_pred: m x 1 vector, label of each sample, can be 0/1
    """
    y_pred = np.zeros(m)
    for i in range(m):
        if final_pred < 0.5:
            y_pred[i] = 0
        else:
            y_pred[i] = 1
    return y_pred

Now we start to use our dataset and construct model.

In [93]:

# Do some data preparation, convert dataframe to numpy array
n_features = X_train.shape[1]

w = np.zeros((1, n_features))

# X_train = X_train.values
# X_test = X_test.values

# y_train = y_train.values
# y_test = y_test.values

m_train =  X_train.shape[0]
m_test =  X_test.shape[0]


Model construction for SGD logistic regression.

In [94]:

#Gradient Descent
coeff_SGD, acc_SGD = LogisticRegressionSGD(w, X_train, y_train, learning_rate=0.0001,num_iterations=2000)
print(acc_SGD)
# TODO: predict probability
final_train_pred_SGD = ...
final_test_pred_SGD = ...
# predict label
y_train_pred_SGD = predict(final_train_pred_SGD, m_train)
y_test_pred_SGD = predict(final_test_pred_SGD, m_test)

print('Optimized weights for SGD', coeff_SGD[:-1])
print('Optimized intercept for SGD', coeff_SGD[-1])

print('Training Accuracy for SGD', accuracy_score(y_train_pred_SGD.T, y_train))
print('Test Accuracy for SGD', accuracy_score(y_test_pred_SGD.T, y_test))

[[-1.261e-01 -3.500e-04 -2.500e-04 -1.002e-01 -1.002e-01 -8.545e-02
  -1.000e-04 -1.500e-04 -5.000e-05 -1.003e-01 -5.000e-05]]
[[ 2522     7     5 ...     1  2006     1]
 [ 6000     6     6 ...     1  2006     1]
 [12735     4     5 ...     1  2008     1]
 ...
 [10880     5     5 ...     1  2008     1]
 [11362     8     5 ...     1  2009     1]
 [ 5000     1     3 ...     1  2007     1]]
[[ -1066.8661   -1443.33825  -2276.3068   -1376.3494   -2116.04355
   -1884.16335  -1581.0073   -1585.9988   -1670.5768   -1761.51795
   -2029.43935  -2125.12905  -2018.5155    -939.1024   -1960.3272
   -2356.62235  -2362.9867   -2601.8427   -1451.10895  -2146.744
   -2144.2417   -1851.86815  -2182.19145  -2076.77555  -1586.4923
   -1999.82585  -3403.81525  -1662.0472   -1997.7992   -1935.57675
   -1261.82945  -2060.55205  -1731.3898   -2219.36355  -2359.67285
   -2788.583    -2084.35715  -1744.78625  -1831.71185  -2091.92355
   -1777.11185  -2206.5719   -1989.8087   -2075.3952   -5420.3689
   -2221.18

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Model construction for AdaGrad logistic regression.

In [ ]:

#AdaGrad Descent
coeff_Ada, acc_Ada = LogisticRegressionSGD(w, X_train, y_train, learning_rate=0.0001,num_iterations=2000)

# TODO: predict probability
final_train_pred_Ada = ...
final_test_pred_Ada = ...
# predict label
y_train_pred_Ada = predict(final_train_pred_Ada, m_train)
y_test_pred_Ada = predict(final_test_pred_Ada, m_test)

print('Optimized weights for Ada', coeff_Ada[:-1])
print('Optimized intercept for Ada', coeff_Ada[-1])

print('Training Accuracy for Ada', accuracy_score(y_train_pred_Ada.T, y_train))
print('Test Accuracy for Ada', accuracy_score(y_test_pred_Ada.T, y_test))

Plot accuracy vs iteration for SGD and AdaGrad. Compare the performance difference. Briefly explain the reason.

In [ ]:

# Plot accuracy vs iteration for SGD and AdaGrad

plt.plot(acc_SGD, label='SGD')
plt.plot(acc_Ada, label='AdaGrad')
plt.ylabel('Accuracy')
plt.xlabel('iterations (per hundreds)')
plt.title('Accuracy improvement over time')
plt.legend(loc='lower right')
plt.show()

### e) Comparision of Scikit, SGD and AdaGrad convergence
Plot the loss function of SGD and AdaGrad over 2000 iterations on both the training and test data. What do you observe? Which one has better accuracy on the test dataset? Why might that be the case?